In [ ]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re

In [ ]:
vvProdutos = 'https://www.vv.com.br/pt/produto/detalhe/'
vieira = 'https://vieirastrieder.com.br/produtos/'
commersul = 'https://www.commersul.com.br/shop/'
sulmatre = 'https://sulmatre.com.br/produtos/'
case = 'https://loja.caseih.com.br/search?text='
nh = 'https://loja.newhollandag.com.br/search?text='
rech = 'https://rech.com/busca?q='

In [ ]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [ ]:
def findOEM(text):
    if 'VV' in text:
        indice = text.find('VV')
        remover = text[indice:]
        text = text.replace(remover, "")
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    remover = [text[:indice], '.', '-']
    for rm in remover:
        text = text.replace(rm, "")
    text = text.replace(' ', "/")
    text = text.split('/')
    lista = []
    for txt in text:
        if txt:
            lista.append(txt)
    for item in lista:
        if not contains_number(item):
            lista.remove(item)
    print(lista)

In [ ]:
testes = ["DEDO CARACOL 1123.464.100.00-VV-0653-01", "DEDO CARACOL 132.0699C1 440.430A1VV-3379", "DEDO CARACOL 3.099.689 VV-0444-00", "DEDO CARACOL 334.972 - VV-0183-01", "DEDO CARACOL 849.80892 - VV-0252-01", "DEDO CARACOL 849.96278 - VV-0278-01", "DEDO CARACOL 849.96279 - VV-0279-01", "DEDO CARACOL H-102.724-VV-0644-01", "DEDO CARACOL H-169.914 - VV-0214-01", "DEDO DUPLO 065.327-VV-0016-02", "DEDO DUPLO 1.307.299.C2 - VV-0060-00", "DEDO DUPLO 3.318.953/379.720 VV-0003-01", "DEDO DUPLO 466.789-VV-0661", "DEDO DUPLO 466.845-VV-0662", "DEDO DUPLO 475.06883 / VV-4057-00", "ANEL SEGURANCA 331.160 ORIG", "BUCHA 849.90897 ORIG", "CABECA NAVALHA 354.413/A", "CUBO 310.049 - 951.1041 ORIG", "DEFLETOR 824.315 / 822.065" ]


In [ ]:
def nh_case(nav, cod, index):
    try:
        btn = nav.find_element(By.CLASS_NAME, 'product-card')
        link = btn.find_element(By.CLASS_NAME, 'stretched-link').get_attribute("href")
        nav.get(link)
        nome = nav.find_element(By.CLASS_NAME, 'product-title__product-name').text
#         nome = re.sub('[0-9]', '', nome)
        codigo = nav.find_element(By.CLASS_NAME, 'product-code').text
        elimg = nav.find_elements(By.CLASS_NAME, 'owl-carousel__images')
        src = []
        for img in elimg:
            img = img.get_attribute('src')
            if 'loja' not in img:
                src.append(img)
        print(len(src))
        src = list(dict.fromkeys(src))
        keys = []
        values = []
        for k in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__title'):
            keys.append(k.text)
        for v in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__value'):
            values.append(v.text)
        dict_infos = dict(zip(keys, values))
        
        print(nome)
        print(codigo)
        print(len(src))
        for img in src:
            print(img)
        print(dict_infos)
    except:
        pass
def nhCase(nav, cod, index):
    try:
        nav.get(str(nh + cod))
        nh_case(nav, cod, index)       
        nav.get(str(case + cod))
        nh_case(nav, cod, index)
    except:
        pass

In [ ]:
def sulmatreSearch(nav, cod, index):
    nav.get(sulmatre)
    elemento = nav.find_element(By.XPATH, '//*[@id="page"]/div[1]/div/section[3]/div/div/div/div/div/section/div/div/div[3]/div/div/div/div/div/div/form/div/div/input[1]')
    elemento.send_keys(cod)
    elemento.send_keys(Keys.ENTER)
    try:
        nome = nav.find_element(By.CLASS_NAME, 'product_title').text
        nome = nome.replace('|', '-')
        elementos = nav.find_elements(By.CLASS_NAME, 'woocommerce-product-gallery__image')
        src = []
        for elemento in elementos:
            img = elemento.find_element(By.TAG_NAME, 'img').get_attribute('src')
            src.append(img)
        keys = []
        values = []
        elementos = nav.find_elements(By.CLASS_NAME, 'woocommerce-product-attributes-item')
        for elemento in elementos:
            keys.append(elemento.find_element(By.CLASS_NAME, 'woocommerce-product-attributes-item__label').text)
            value = elemento.find_element(By.CLASS_NAME, 'woocommerce-product-attributes-item__value')
            values.append(value.find_element(By.TAG_NAME, 'p').text)
        dict_infos = dict(zip(keys, values))
        
        print(nome)
        for img in src:
            print(img)
        print(dict_infos)
    except:
        pass        

In [ ]:
def rechSearch(nav, cod, index):
    elemento = WebDriverWait(nav, 10).until(EC.presence_of_element_located((By.ID, 'search-input')))
    elemento.send_keys(str(cod))
    elemento.send_keys(Keys.ENTER)
    try:
        elemento = nav.find_element(By.ID, 'showcase')
        elementos = elemento.find_elements(By.CLASS_NAME, 'y-item-wrapper-showcase')
        urls = []
        for elemento in elementos:
            url = elemento.find_element(By.TAG_NAME, 'link').get_attribute('href')
            urls.append(url)
            print(url)
        for url in urls:
            src = []
            keys = []
            values = []
            nav.get(str(url))
            nav.implicitly_wait(0.8)
            elementos = nav.find_elements(By.CLASS_NAME, 'zoomImg')
            for elemento in elementos:
                src.append(elemento.get_attribute('src'))
            nome = nav.find_element(By.CLASS_NAME, 'product__name').text
            codigo = nav.find_element(By.CLASS_NAME, 'product__partcode').text
            descricao = nav.find_element(By.CLASS_NAME, 'panel-description').text
            elemento = nav.find_element(By.CLASS_NAME, 'table-specifications')
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr th')
            for k in elementos:
                keys.append(k.text)
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr td')
            for v in elementos:
                values.append(v.text)
            dict_infos = dict(zip(keys, values))
            for x in dict_infos.keys():
                y = dict_infos.get(x)
                y = y.replace('\n', ', ')
                dict_infos.update({x : y})
            print(nome)
            print(codigo)
            print(descricao)
            print(dict_infos)
            
    except:
        pass
            

In [ ]:
def loginRech(nav):
    nav.get('https://www.rech.com/login?targetUri=%2F')
    elemento = nav.find_element(By.ID, 'username')
    elemento.send_keys('yuri@cotripal.com.br')
    elemento = nav.find_element(By.ID, 'password')
    elemento.send_keys('Testes1!')
    elemento = nav.find_element(By.ID, 'submit')
    elemento.click()
    nav.implicitly_wait(0.8)
    rechSearch(nav, 87041572, 1)

In [ ]:
def commersulSearch(nav, cod, index):
    nav.get(str(commersul))
    elemento = nav.find_element(By.XPATH, '//*[@id="th-search-form-1"]')
    elemento.send_keys(str(cod))
    elemento.send_keys(Keys.ENTER)
    nav.implicitly_wait(0.8)
    try:
        nome = nav.find_element(By.CLASS_NAME, 'product_title').text
        dict_infos = {
            nav.find_element(By.XPATH, '//*[@id="product-14471"]/div[1]/div/div[6]/div/div[1]/table/tbody/tr[2]/th').text : nav.find_element(By.XPATH, '//*[@id="product-14471"]/div[1]/div/div[6]/div/div[1]/table/tbody/tr[2]/td').text,
            nav.find_element(By.XPATH, '//*[@id="product-14471"]/div[1]/div/div[6]/div/div[1]/table/tbody/tr[3]/th').text : nav.find_element(By.XPATH, '//*[@id="product-14471"]/div[1]/div/div[6]/div/div[1]/table/tbody/tr[3]/td').text,
            nav.find_element(By.XPATH, '//*[@id="product-14471"]/div[1]/div/div[6]/div/div[1]/table/tbody/tr[4]/th').text : nav.find_element(By.XPATH, '//*[@id="product-14471"]/div[1]/div/div[6]/div/div[1]/table/tbody/tr[4]/td').text
        }
        elemento = nav.find_element(By.XPATH, '//*[@id="product-14471"]/div[1]/div/div[3]/div/div/figure/div/a')
        src = elemento.get_attribute('href')
        print(src)
        print(nome)
        print(dict_infos)
    except:
        pass

# Commersul
###### produto de exemplo: 756397
buscar por id 'th-search-form-2'
enviar codigo
apertar enter
se encontrar o produto entra direto
try buscar informações do produto
except pass

*nome* buscar por classe 'product_title'
*elemento* buscar por classe 'th-product__meta'
*keys* dentro do elemento buscar por tag 'th'
*values* dentro do elemento buscar por tag 'td'
depois de o dicionário estiver pronto usar .pop('key') para remover codigo commersul
> imagem (por via das dúvidas considerar que tem mais de 1)

buscar por classe 'zoomImg'

# MCL
###### Produto de exemplo: 824367


In [ ]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)
# produtos = dict(zip(query['ID'], query['Produto']))

# nhCase(nav, '44906471', '1')
# sulmatreSearch(nav, '87617623', '1')
# loginRech(nav)
commersulSearch(nav, 756397, 1)

755854
825727
756397
290171
824495
417722
394027

DQ21353
DQ09620
58020015
0521019147
2728775
7010041276
80362075
H205344
84981654
MCL002034
30020089MCL
1982672C1
15240019
60071662
DQ18156L
1308425C1
MCL001056
AN273676
CQ50545
CQ10525
H221045
H177242
87457505 - 1322467C5
322901
290103
DQ13118
70529432N
214500C2
9577126
74104102
3469577
87552399
020800057 +15
MCL4044


In [ ]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options)
nav.get('https://mcl.ind.br/produtos')

In [ ]:
teste2 = ['ALAVANCA 849.98670 ORIG', 'AMORTECEDOR 755.833-B - 47509', 'AMORTECEDOR 849.95693/950.9946 ORIG', 'ARRUELA COBRE 26 MM 465.374 - 756.853']
teste3 = ['320.053 - 373.929 - VV-0393-00', 'PORCA CASTELO 412.351 ORIG', 'PORCA CASTELO 20.0130/100', 'PORCA CASTELO 82.0040 ORIG', 'PORCA CASTELO 15.04.00.15', 'PORCA CASTELO 35.04.00.02', 'PORCA SEXT MB-20 82.03.06.20 CASTELO', 'PORCA CASTELO M-20 YL-03-01-20', 'PORCA CASTELO M30 72-03-02-30', 'PORCA CASTELO MB 14X1.5 YL-03-01-14', 'PORCA CASTELO MB 18X1.5 YL-03-01-18', 'PORCA CASTELO M20 95012101 / 920522022', 'PORCA CASTELO M24 61932212', 'PORCA CASTELO M27 61832412', 'PORCA CASTELO M30 61832612', 'PORCA CASTELO 612580', 'PORCA CASTELO 000 937 0300 - 140.213-7', 'PORCA CASTELO 000 990 1955 - 318.046-8', 'PORCA CASTELO 50550/00 - 50.04', 'PORCA CASTELO 50666/00']
for teste in teste3:
    findOEM(teste)